# SIO 113: Assignment #9

#### Instructor: Dave May (dmay@ucsd.edu)

#### Assistant: Gabrielle Hobson (ghobson@ucsd.edu)

#### Scripps Institution of Oceanography, UCSD.

----

<div class="alert alert-block alert-info"><b>Preliminary:</b>
Create a new notebook and rename it using the format YourLastname_FirstInitial_HW_09. 
</div>

<div class="alert alert-block alert-warning"><b>Note:</b>
This assignment requires the following packages / modules / methods
    
`
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc4
import dem_utils as dem_utils
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
`    
</div>

## 1. Plotting terrain contours
1. Read in the data file `Datasets/topodata.csv` into a Pandas `DataFrame`. These are data from the etopo data that we have used on the California coastline.
2. Use the `.unique()` method on the lats and lons `Series` and the `len()` function to find the number of unique latitudes and longitudes.
3. Turn each of the columns into numpy arrays, and reshape them to be `(n_lats, n_lons)` where `n_lats` is the number of unique latitudes and `n_lons` is the number of unique longitudes. Note: They must be in this order or this will not work.
4. Using cartopy, plot a lambert conformal map with central longitude -120 and central latitude 35. Use `ax.set_extent()` to set the axis extents to the minimum and maximum latitudes and longitudes in your `DataFrame`.
5. Use `plt.contourf()` to plot your topography data on the map. Use the `transform` argument to plot your contours in the right place. Use the arguments `cmap='terrain'` and `levels=30` to make the map look nice (if you don't like the color scale, you can use the `vmin` and `vmax` arguments to change the maximum and minimum contour levels).

## 2. 3D topographic plotting

1. Following the lecture on digital elevation models, load the topography dataset of the Rocky's `Datasets/topo_co.nc` using NetCDF. The dataset differs from the lecture, but the procedure is the same.
The variable named `'elevation'` read from the NetCDF file will need to "reshaped" using `np.reshape()` as per the lecture.
2. Create a meshgrid object using `lon_1d`, `lat_1d` using `np.meshgrid()`.
3. Using `from mpl_toolkits.mplot3d import Axes3D`, create a 3D interactive plot using *both*
`ax.plot_surface()` and `ax.contour3D()` *in the same plot*. Make the surface transparent via the `alpha` argument and set its color to grey or black. Plot 20 evenly spaced contours from the range $z = [0, 3500]$ m. Color the contours using the color map created by `dem_utils.create_topo_cmap()`. Add a color bar to your plot, label the axis and include the units of all physical quantities.
4. In part 3, it is likely the result is hard to interpret due to the high frequency variations of the topography. For the purpose of visualization, it is sometimes desirable to smooth the data a little - just for the sake of making a nicer picture. One way to do that is using Gaussian blurring. To apply Gaussian blur use the code snippet shown below. This code will process `z_2d` and return a blurred version `z_2d_b`. Re-do part 3 using `z_2d_b` and compare the results.

```python
# Apply Gaussian blur
from scipy.ndimage import gaussian_filter

z_2d_b = gaussian_filter(z_2d, sigma=4)
```

<div class="alert alert-block alert-danger"><b> 
To receive full credit, your notebook:
    
1. Must have the correct name;
2. Must be fully commented;
3. Must run as expected;
4. Must be submitted into Canvas before the deadline.
</b></div>